In [1]:
pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=e3102e18b779cfd66c36f21c5bb6313d331c91843da816efbeb75585c84e23c6
  Stored in directory: /root/.cache/pip/wheels/83/71/2b/40d17d21937fed496fb99145227eca8f20b4891240ff60c86f
Successfully built sentence-transformers
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import os
from keras import layers, models
from sentence_transformers import SentenceTransformer
import pandas as pd
from sklearn.model_selection import train_test_split
encoder = SentenceTransformer('bert-base-nli-mean-tokens')

In [3]:
df = pd.read_csv("/kaggle/input/emotion-detection-from-text/tweet_emotions.csv")

In [4]:
df

,tweet_id,sentiment,content
0,1956967341,empty,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,wants to hang out with friends SOON!
4,1956968416,neutral,@dannycastillo We want to trade with someone w...
...,...,...,...
39995,1753918954,neutral,@JohnLloydTaylor
39996,1753919001,love,Happy Mothers Day All my love
39997,1753919005,love,Happy Mother's Day to all the mommies out ther...
39998,1753919043,happiness,@niariley WASSUP BEAUTIFUL!!! FOLLOW ME!! PEE...


In [7]:
hm = {'anger': 2,
'boredom': 0,
'empty': 2,
'enthusiasm': 1,
'fun': 1,
'happiness': 1,
'hate': 2,
'love': 1,
'neutral': 0,
'relief': 1,
'sadness': 2,
'surprise': 1,
'worry': 2}

In [13]:
text = df[['content']]
train = []
classification = []
for y in range(40000):
    train.append(text['content'][y])
label = df[['sentiment']]
for y in range(40000):
    classification.append(hm[label['sentiment'][y]])

In [14]:
print(train[69], classification[69])

So great to see Oin &amp; Cynthia.  So happy.  Dinner was great, cute little place.  Too bad Oin got sick afterwards. 1


In [15]:
train = encoder.encode(train)

Batches:   0%|          | 0/1250 [00:00<?, ?it/s]

In [16]:
train.shape

(40000, 768)

In [17]:
x_train, x_test, y_train, y_test = train_test_split(train, classification, train_size=0.8)

In [20]:
y_train = np.array(y_train)
y_test = np.array(y_test)

In [26]:
def build_model(training, classification, ep):
    temp_model = models.Sequential([
        layers.Input(shape=(768)),
        #layers.Dense(1000, activation="relu"),
        layers.Dropout(0.3),
        layers.Dense(1000, activation="relu"),
        layers.Dropout(0.56),
        layers.Dense(700, activation="relu"),
        layers.Dropout(0.2),
        layers.Dense(500, activation='relu'),
        layers.Dropout(0.4),
        layers.Dense(200, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(100, activation="relu"),
        layers.Dropout(0.3),
        layers.Dense(3, activation="sigmoid")
    ])
    temp_model.compile(optimizer='adam', loss="sparse_categorical_crossentropy", metrics=['accuracy'])
    #x_train, x_test, y_train, y_test = train_test_split(training, classification, train_size=0.9)
    temp_model.fit(training, classification, epochs=ep)
    #temp_model.evaluate(x_test, y_test)
    return temp_model
    #temp_model.save("/kaggle/working/" + feeling + ".h5")

In [ ]:
m1 = build_model(x_train, y_train, 30)

Epoch 1/30
1000/1000 [==============================] - 6s 3ms/step - loss: 0.9283 - accuracy: 0.5898
Epoch 2/30
1000/1000 [==============================] - 3s 3ms/step - loss: 0.9041 - accuracy: 0.6082
Epoch 3/30
1000/1000 [==============================] - 3s 3ms/step - loss: 0.8934 - accuracy: 0.6116
Epoch 4/30
1000/1000 [==============================] - 3s 3ms/step - loss: 0.8896 - accuracy: 0.6151
Epoch 5/30
1000/1000 [==============================] - 4s 4ms/step - loss: 0.8892 - accuracy: 0.6133
Epoch 6/30
1000/1000 [==============================] - 3s 3ms/step - loss: 0.8834 - accuracy: 0.6181
Epoch 7/30
1000/1000 [==============================] - 3s 3ms/step - loss: 0.8835 - accuracy: 0.6181
Epoch 8/30
1000/1000 [==============================] - 3s 3ms/step - loss: 0.8834 - accuracy: 0.6154
Epoch 9/30
  52/1000 [>.............................] - ETA: 2s - loss: 0.8447 - accuracy: 0.6346

In [23]:
m1.evaluate(x_test, y_test)

250/250 [==============================] - 1s 2ms/step - loss: 0.8696 - accuracy: 0.6199


[0.8695597648620605, 0.6198750138282776]